In [13]:
import json
import pandas as pd
import numpy as np
from pprint import pprint
import re

In [22]:
def json_parse(lmi_json):
    with open(lmi_json) as json_file:
         json_data = json.load(json_file)
    return json_data

json_dict = json_parse('infile.json')
pprint(json_dict)

{'ads': 0,
 'partials': {'footer_0': {'description': 'Refresh dfp and third-party ads',
                           'name': 'ads/refresher',
                           'vars': {'amazon_page_depth_refresh_start': '0',
                                    'dfp_refresh_interval': '7000',
                                    'dfp_timer_refresh_interval': '1500',
                                    'max_dfp_consecutive_timed': '2',
                                    'max_third_party_consecutive_timed': '10',
                                    'refresh_event_list': 'carousel_slide_change',
                                    'refresh_on_every_nth_click': '1',
                                    'third_party_refresh_interval': '7000'}},
              'footer_1': {'name': 'cyclone/footer2',
                           'vars': {'show_credit': 'off'}},
              'footer_2': {'name': 'tracking/pagebottom',
                           'vars': {'comscore': 'on',
                                   

In [57]:
#Set constants

OpenX_ids = {"728": "537088038" , "160": "537088042" , "300": "537088040"}
Casale_ids = {"728": "154714", "160": "154714", "300": "154714"}
ids = [OpenX_ids, Casale_ids]

#Make sure IDs are strings
for i in ids:
    for key in i:
        i[key] = str(i[key])



True

# functions

In [15]:
def lmiTagging(json, tag, openx_ids, casale_ids):
    if tag[:2] == "gu":
            replaceDFPUnits_Mobile(json,tag)
    elif tag[:2] == "ar":
        replaceDFPUnits_Desktop(json, tag, openx_ids, casale_ids)

#Mobile Functions
'''Replaces ad units in '''
def replaceDFPUnits_Mobile(json, tag):
    tag = tagging(tag)
    if len(tag.group()) <= 7:
        adunitPre = "51859126/parent_m_ar" + "/" + str(tag.group())
    else:
        adunitPre = "51859126/parent_m_ar_" + str(tag.group(1)) + "/" + str(tag.group())
    for module in json['partials']:
        name = json['partials'][module]['name']
        if name == 'ads/dfpslot' and json['partials'][module]['vars']['dfp_height'] != '1' :
            json['partials'][module]['vars']['dfp_ad_unit'] = adunitPre + str(determineAdSlot(json['partials'][module]['vars']['dfp_ad_unit']))
        elif name == 'ads/rubicon':
            json['partials'][module]['vars']['rp_kw'] = str(tag.group()) + "_5"
        elif name == 'ads/outbrain':
            json['partials'][module]['vars']['variant'] = str(tag.group()) + "_1"
    return json

#Desktop Functions
#/51859126/parent_ar_roa/child_ar_roa_2/roa2_728x90_2
#/51859126/parent_ar/child_ar_60/ar60_160x600_1
#/51859126/parent_m_ar_roa/gut_roa_m_25_1
#/51859126/parent_m_ar/gut_825_1

def replaceDFPUnits_Desktop(json, tag, siteCode, openx_ids, casale_ids):
    #replaces all ad units with a given tag
    tag = tagging(tag)
    for module in json['partials']:
        if len(tag.group()) <= 5:
            adunitPre = "51859126/parent_ar" + "/" + "child_ar_" + str(tag.group(2)) + "/" + str(tag.group())
        else:
            adunitPre = "51859126/parent_ar_" + str(tag.group(3)) + "/" + "child_ar_" + str(tag.group(3)) + "_" + str(tag.group(5)) + "/" + str(tag.group(2))
        name = json['partials'][module]['name']
        ad_unit = json['partials'][module]
        #Replace DFP ad units
        if name == 'ads/dfpslot':
            ad_unit['vars']['dfp_ad_unit'] = adunitPre + "_" + str(determineAdSize(json['partials'][module]['vars']['dfp_width'])) + str(determineAdSlot(json['partials'][module]['vars']['dfp_ad_unit']))
        elif name == 'ads/outbrain':
            json['partials'][module]['vars']['variant'] = str(tag) + "_1"
        elif name == 'links/openx':
            replaceOpenX(module, openx_ids)
        elif name == 'links/casale':
            replaceCasale(module, casale_id)
        elif name == 'video/universal':
            json['partials'][module]['vars']['ad_tag'] = 'http://vast.optimatic.com/vast/getVast.aspx?id=a253n547s864w&zone=' + str(tag) + '&pageURL=__page-url__&pageTitle=__video-title__&cb=__random-number'
    return json

#Helper functions  

"""The tagging function breaks up a given tag into components so that other functions can access information about the site and tag.
   Tags take the following four forms:
   1) gut_XXX_m_YYY --> XXX: group(2), YYY: group(4)
   2) ar_XXXYYY --> XXX: group(3), YYY: group(5) XXXYYY: group(2)
   3) gut_YYY --> YYY: group(1)
   4) arYYY --> YYY: group(2)
   
   The entire tag can be accessed by calling .group() on the regex object. Accessing different parts of the tag can be done by referencing the mapping above
"""
def tagging(tag):
    if tag[:3] != "gut":
        tagRegexD = re.compile(r'ar(_?)(((\w\w\w)?)((\d?)(\d?)(\d?)))')
        tagging = tagRegexD.search(tag)
    else:
        tagRegex = re.compile(r'gut_(\w\w\w?)((_m_)?)((\d?)(\d?)(\d?))')
        tagging = tagRegex.search(tag)
    return tagging
    

def determineAdSlot(ad_unit):
    end = ad_unit[-2:] 
    return end
    
def determineAdSize(ad_unit):
    if ad_unit == '300':
        return "300x250"
    elif ad_unit == '728':
        return "728x90"
    elif ad_unit == '160':
        return "160x600"
    
def replaceDFP_Desktop(module):
    if json['partials'][module]['name'] == 'ads/dfpslot':
        ad_unit['dfp_ad_unit'] = "51859126/parent_ar/" + "child_ar_" + str(tag[-2:]) + "/" + str(tag) + "_" + str(determineAdSize(varz['dfp_width']))
        
def replaceDFP_Mobile(module):
    if module['name'] == 'ads/dfpslot':
        module['dfp_ad_unit'] = "51859126/parent_m_ar/" + str(tag) + str(determineAdSlot(json['partials'][module]['vars']['dfp_ad_unit']))

def replaceOB(module):
    if module['name'] == 'ads/outbrain':
        module['vars']['variant'] = str(tag) + "_1"

def replaceCasale(module, casale_id):
    varz = json['partials'][module]['vars']
    if varz['width'] == '728':
        varz['casale_id'] = casale_ids["728"]
    elif varz['width'] == '160':
        varz['casale_id'] = casale_ids["160"]
    elif varz['width'] == '300':
        varz['casale_id'] = casale_ids["300"]
    return varz
        
def replaceOpenX(varz, OpenX_ids):
    varz = json['partials'][module]['vars']
    if varz['width'] == '728':
        varz['auid'] = openx_ids["728"]
    elif varz['width'] == '160':
        varz['auid'] = openx_ids["160"]
    elif varz['width'] == '300':
        varz['auid'] = openx_ids["300"]
    return varz
                
    
#QA Functions   
def QA_div_id():
     for module in json['partials']:
        if json['partials'][module]['name'] == 'ads/dfpslot':
            json['partials'][module]['vars'] = tag
            
   

In [385]:
tag = tagging("gut_roa_m_10")
tag.group(1)

'roa'

In [355]:
json_data4 = replaceDFPUnits_Desktop2(json_dict,"ar50","",Casale_ids,OpenX_ids)
pprint(json_data4)

{'ads': 0,
 'adsnip': [],
 'compactor': 'on',
 'partials': {'centerTop_0': {'instance': 'gute_slideshow',
                              'name': 'utility/wrapper'},
              'centerTop_1': {'description': 'Includes Facebook, Pinterest, '
                                             'Linkedin, Reddit, Twitter, '
                                             'Google+, StumbleUpon, and Tumblr',
                              'name': 'social/share/box',
                              'vars': {'enable_googleplus': 'no',
                                       'enable_twitter': 'no',
                                       'heading': 'Share this article',
                                       'label': 'long'}},
              'centerTop_10': {'instance': 'close',
                               'name': 'utility/wrapper',
                               'vars': {'type': 'close'}},
              'centerTop_11': {'description': 'this module is used to set up '
                                     

In [23]:
#new_json = open('json_out','w')
json_data = replaceDFPUnits_Mobile(json_dict,"gut_roa_m_9")
pprint(json_data)

with open('outfile.json', 'w') as outfile:
    json.dump(json_data, outfile)

{'ads': 0,
 'partials': {'footer_0': {'description': 'Refresh dfp and third-party ads',
                           'name': 'ads/refresher',
                           'vars': {'amazon_page_depth_refresh_start': '0',
                                    'dfp_refresh_interval': '7000',
                                    'dfp_timer_refresh_interval': '1500',
                                    'max_dfp_consecutive_timed': '2',
                                    'max_third_party_consecutive_timed': '10',
                                    'refresh_event_list': 'carousel_slide_change',
                                    'refresh_on_every_nth_click': '1',
                                    'third_party_refresh_interval': '7000'}},
              'footer_1': {'name': 'cyclone/footer2',
                           'vars': {'show_credit': 'off'}},
              'footer_2': {'name': 'tracking/pagebottom',
                           'vars': {'comscore': 'on',
                                   